In [83]:
# === Imports ===
import os, re, math, json, random
import numpy as np
import pandas as pd

from biom import load_table

from sklearn.model_selection import StratifiedShuffleSplit, GroupShuffleSplit
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.inspection import permutation_importance

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline as ImbPipeline

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

# === Фолдеры, классы ===
DATA_DIRS = {
    "agricultural": "biom_data/agricultural",
    "desert":       "biom_data/desert",
    "forest":       "biom_data/forest",
    "grassland":    "biom_data/grassland",
    "tropical_rainforest": "biom_data/tropical_rainforest",
}

# === Пороговые параметры отбора фич ===
PREVALENCE_MIN     = 0.02       # мин. доля сэмплов класса, где таксон встречается (для CAP — считаем по train!)
MEAN_ABUND_MIN     = 1e-5       # мин. средняя ненулевая относительная абунданса (по train)
CAP_MAX_FEATURES   = 5000       # глобальный "потолок" числа фич после CAP-фильтра (по train)
MIN_CLASS_PREV     = 0.05       # мин. доля встречаемости внутри класса (по train)
TOP_PER_CLASS      = 300        # сколько топ таксонов брать на класс (по train)
K_TOTAL            = 1500       # итоговый потолок признаков после MI-ужатия (по train)

TEST_SIZE          = 0.20

# Модели
MODELS = {
    "LogReg": LogisticRegression(max_iter=2000, n_jobs=None, random_state=RANDOM_STATE, C=1.0),
    "RF":     RandomForestClassifier(n_estimators=400, max_depth=None, random_state=RANDOM_STATE, n_jobs=-1),
    "SVMrbf": SVC(C=2.0, gamma="scale", probability=True, random_state=RANDOM_STATE),
    "KNN5":   KNeighborsClassifier(n_neighbors=5),
    "GBC":    GradientBoostingClassifier(random_state=RANDOM_STATE)
}



In [84]:
def load_biom_table(path):
    table = load_table(path)
    df = table.to_dataframe(dense=True)  # rows: obs, cols: samples
    # нормализуем имена
    df.index = df.index.astype(str)
    df.columns = df.columns.astype(str)
    return df

all_mats = []
meta_rows = []

for cls, folder in DATA_DIRS.items():
    if not os.path.exists(folder):
        print(f"⚠️ Нет папки: {folder}")
        continue
    for fn in sorted(os.listdir(folder)):
        if not fn.endswith(".biom"):
            continue
        file_path = os.path.join(folder, fn)
        tbl = load_biom_table(file_path)
        n_obs, n_samp = tbl.shape
        file_stem = os.path.splitext(fn)[0]
        # добавим префикс к именам сэмплов: Class:FileStem:SampleId
        cols_map = {c: f"{cls}:{file_stem}:{c}" for c in tbl.columns}
        tbl.rename(columns=cols_map, inplace=True)

        all_mats.append(tbl)
        meta_rows.append({
            "class": cls,
            "file": fn,
            "file_stem": file_stem,
            "n_obs": n_obs,
            "n_samples": n_samp
        })

summary = pd.DataFrame(meta_rows)
print("Файлов:", len(summary))
display(summary.head())

# таксоны × сэмплы (объединяем по таксонам)
all_data = pd.concat(all_mats, axis=1).fillna(0.0)
all_data = all_data.groupby(all_data.index).sum()
print("Форма all_data:", all_data.shape)













Файлов: 598


,class,file,file_stem,n_obs,n_samples
0,agricultural,agricultural_1.biom,agricultural_1,274,1
1,agricultural,agricultural_10.biom,agricultural_10,10,1
2,agricultural,agricultural_100.biom,agricultural_100,686,1
3,agricultural,agricultural_11.biom,agricultural_11,19,1
4,agricultural,agricultural_12.biom,agricultural_12,313,1


Форма all_data: (12089, 598)


In [85]:
# суммы по сэмплам
sample_sums = all_data.sum(axis=0)
sample_sums[sample_sums == 0] = 1.0  # защита от деления на ноль

# относительные абундансы
wide_rel = (all_data / sample_sums)

# индексы: сэмплы
wide_rel = wide_rel.T  # теперь rows = samples, cols = taxa
wide_rel.index.name = "sample"

# метки классов
def parse_class(s):
    # ожидаем формат: class:file_stem:orig_sample
    return s.split(":", 1)[0]

# группа-источник (file_stem) для группового сплита
def parse_group(s):
    # class:file_stem:...
    parts = s.split(":")
    return parts[1] if len(parts) > 1 else "ungrouped"

y_full = wide_rel.index.to_series().apply(parse_class)
groups = wide_rel.index.to_series().apply(parse_group)

print("Классы (всего):", y_full.value_counts().to_dict())
print("Групп (file_stem):", len(groups.unique()))





Классы (всего): {'forest': 200, 'agricultural': 100, 'grassland': 100, 'tropical_rainforest': 100, 'desert': 98}
Групп (file_stem): 598


In [86]:
use_groups = True  # если хочешь без групп — поставь False

if use_groups:
    gss = GroupShuffleSplit(n_splits=1, test_size=TEST_SIZE, random_state=RANDOM_STATE)
    tr_idx, te_idx = next(gss.split(wide_rel, y_full, groups=groups))
else:
    sss = StratifiedShuffleSplit(n_splits=1, test_size=TEST_SIZE, random_state=RANDOM_STATE)
    tr_idx, te_idx = next(sss.split(wide_rel, y_full))

X_train0 = wide_rel.iloc[tr_idx].copy()
X_test0  = wide_rel.iloc[te_idx].copy()
y_train0 = y_full.iloc[tr_idx].copy()
y_test0  = y_full.iloc[te_idx].copy()

print("Train shape:", X_train0.shape, " Test shape:", X_test0.shape)
print("Train class counts:", y_train0.value_counts().to_dict())
print("Test  class counts:", y_test0.value_counts().to_dict())



Train shape: (478, 12089)  Test shape: (120, 12089)
Train class counts: {'forest': 160, 'grassland': 85, 'desert': 79, 'agricultural': 77, 'tropical_rainforest': 77}
Test  class counts: {'forest': 40, 'agricultural': 23, 'tropical_rainforest': 23, 'desert': 19, 'grassland': 15}


In [87]:
# prevalence по train: доля сэмплов с ненулём
prev = (X_train0 > 0).mean(axis=0)  # по колонкам (таксонам)

# средняя ненулевая относительная абунданса по train
def nonzero_mean(col):
    nz = col[col > 0]
    return float(nz.mean()) if len(nz) else 0.0

mean_abund = X_train0.apply(nonzero_mean, axis=0)

cap_mask = (prev >= PREVALENCE_MIN) & (mean_abund >= MEAN_ABUND_MIN)
cap_features = mean_abund[cap_mask].sort_values(ascending=False).index.tolist()

if len(cap_features) > CAP_MAX_FEATURES:
    cap_features = cap_features[:CAP_MAX_FEATURES]

print(f"CAP features: {len(cap_features)}")

X_tr = X_train0[cap_features].copy()
X_te = X_test0[cap_features].copy()
y_tr = y_train0.copy()
y_te = y_test0.copy()


CAP features: 2632


In [88]:
# 1) доля встречаемости внутри каждого класса по train
class_pools = []
for cls in sorted(y_tr.unique()):
    cls_mask = (y_tr == cls)
    X_cls = X_tr.loc[cls_mask]
    prev_cls = (X_cls > 0).mean(axis=0)
    # отберём по порогу и отсортируем по средней ненулевой абундансе
    nnz_mean_cls = X_cls.apply(lambda c: c[c > 0].mean() if (c > 0).any() else 0.0, axis=0)
    mask_cls = prev_cls >= MIN_CLASS_PREV
    feats_cls = nnz_mean_cls[mask_cls].sort_values(ascending=False).index.tolist()
    feats_cls = feats_cls[:TOP_PER_CLASS]
    class_pools.extend(feats_cls)

pool = sorted(set(class_pools))
print(f"Pool after per-class selection: {len(pool)}")

X_tr_pool = X_tr[pool].copy()
X_te_pool = X_te[pool].copy()

# 2) mutual information для ужатия до K_TOTAL — СТРОГО по train
le = LabelEncoder()
y_enc = le.fit_transform(y_tr)

mi = mutual_info_classif(X_tr_pool.fillna(0.0), y_enc, random_state=RANDOM_STATE, discrete_features=False)
mi_s = pd.Series(mi, index=X_tr_pool.columns).sort_values(ascending=False)

chosen = mi_s.index[:min(K_TOTAL, len(mi_s))].tolist()
print("Chosen features:", len(chosen))

X_train = X_tr_pool[chosen].fillna(0.0)
X_test  = X_te_pool[chosen].fillna(0.0)
y_train = y_tr.copy()
y_test  = y_te.copy()


Pool after per-class selection: 934
Chosen features: 934


In [89]:
reports = {}

for name, model in MODELS.items():
    # Скейлер полезен для LogReg/SVM/KNN; для RF/GBC он не навредит,
    # т.к. мы скейлим внутри пайплайна (fit только на train)
    pipe = ImbPipeline(steps=[
        ("ros",    RandomOverSampler(random_state=RANDOM_STATE)),
        ("scaler", StandardScaler(with_mean=True, with_std=True)),
        ("clf",    model)
    ])

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    print(f"\n=== {name} ===")
    print("Accuracy:", f"{acc:.4f}")
    print(classification_report(y_test, y_pred, digits=4))
    reports[name] = acc



=== LogReg ===
Accuracy: 0.9750
                     precision    recall  f1-score   support

       agricultural     0.9583    1.0000    0.9787        23
             desert     1.0000    1.0000    1.0000        19
             forest     0.9756    1.0000    0.9877        40
          grassland     1.0000    0.9333    0.9655        15
tropical_rainforest     0.9545    0.9130    0.9333        23

           accuracy                         0.9750       120
          macro avg     0.9777    0.9693    0.9730       120
       weighted avg     0.9752    0.9750    0.9747       120


=== RF ===
Accuracy: 0.9833
                     precision    recall  f1-score   support

       agricultural     1.0000    1.0000    1.0000        23
             desert     1.0000    1.0000    1.0000        19
             forest     0.9524    1.0000    0.9756        40
          grassland     1.0000    0.9333    0.9655        15
tropical_rainforest     1.0000    0.9565    0.9778        23

           accurac

In [ ]:
# Выбери лучшую модель по accuracy для примера важности
best_name = max(reports, key=reports.get)
best_model = MODELS[best_name]

pipe_best = ImbPipeline(steps=[
    ("ros",    RandomOverSampler(random_state=RANDOM_STATE)),
    ("scaler", StandardScaler(with_mean=True, with_std=True)),
    ("clf",    best_model)
])

pipe_best.fit(X_train, y_train)

perm = permutation_importance(
    pipe_best,
    X_test,
    y_test,
    n_repeats=10,
    random_state=RANDOM_STATE
)

imp = pd.Series(perm.importances_mean, index=X_test.columns).sort_values(ascending=False)
display(imp.head(30))
